In [5]:
import random
import functools
import operator

# --- Configuration (Hamming(16, 11)) ---
N_DATA = 11
N_TOTAL = 16
# 0-based indices for parity bits P1, P2, P4, P8, P16
P_IDX = [0, 1, 3, 7, 15]
# 0-based indices for data bits
D_IDX = [i for i in range(N_TOTAL) if i not in P_IDX]


CHECK_INDICES = {
    0: [0, 2, 4, 6, 8, 10, 12, 14],  # P1 (Index 0) checks positions 1, 3, 5...
    1: [1, 2, 5, 6, 9, 10, 13, 14],  # P2 (Index 1) checks positions 2, 3, 6...
    3: [3, 4, 5, 6, 11, 12, 13, 14], # P4 (Index 3) checks positions 4, 5, 6...
    7: [7, 8, 9, 10, 11, 12, 13, 14] # P8 (Index 7) checks positions 8, 9, 10...
}

# XOR helper
xor_sum = lambda bits: functools.reduce(operator.xor, bits, 0)


def encode(data_bits):
    """Encodes 11 data bits into a 16-bit Hamming codeword."""
    codeword = [0] * N_TOTAL
    # 1. Place data
    for i in range(N_DATA):
        codeword[D_IDX[i]] = data_bits[i]

    # 2. Calculate P1, P2, P4, P8
    for p_idx in P_IDX[:-1]: # Indices 0, 1, 3, 7
        bits_to_xor = [codeword[i] for i in CHECK_INDICES[p_idx]]
        codeword[p_idx] = xor_sum(bits_to_xor)

    # 3. Calculate P16 (overall parity of bits 0-14)
    codeword[15] = xor_sum(codeword[:-1])
    return codeword

def check_correct(received_code):
    """Checks a 16-bit codeword, identifies/corrects single errors."""
    syndrome_val = 0
    # 1. Calculate syndrome bits (c1, c2, c4, c8 -> contribute 1, 2, 4, 8 to syndrome)
    for i, p_idx in enumerate(P_IDX[:-1]): # Indices 0, 1, 3, 7
        bits_to_xor = [received_code[k] for k in CHECK_INDICES[p_idx]]
        if xor_sum(bits_to_xor) != 0: # Check failed for this parity bit
            syndrome_val += (2**i)

    # 2. Check overall parity (P16 check)
    overall_parity_ok = (xor_sum(received_code) == 0)

    # 3. Determine error location (1-based)
    error_pos = 0 # 0 = no error, -1 = double error, >0 = single error pos
    corrected_code = list(received_code) # Copy for potential correction

    if syndrome_val == 0:
        if not overall_parity_ok:
            error_pos = 16 # Error in P16 itself
    else: # syndrome_val > 0
        if not overall_parity_ok:
            error_pos = syndrome_val # Single error at syndrome location
        else:
            error_pos = -1 # Double error detected

    # 4. Correct if single error found
    if error_pos > 0:
        err_idx_0based = error_pos - 1
        corrected_code[err_idx_0based] = 1 - corrected_code[err_idx_0based]

    return error_pos, corrected_code


# 1. Generate Data
data = random.choices([0, 1], k=N_DATA)
print(f"Data:     {''.join(map(str, data))}")

# 2. Encode
encoded = encode(data)
print(f"Encoded:  {''.join(map(str, encoded))}")

# 3. Introduce Error
error_pos_actual = random.randint(1, N_TOTAL)
corrupted = list(encoded)
corrupted[error_pos_actual - 1] = 1 - corrupted[error_pos_actual - 1]
print(f"Corrupted:{''.join(map(str, corrupted))} (err @{error_pos_actual})")

# 4. Check and Correct
error_pos_found, corrected = check_correct(corrupted)

# 5. Report Results
if error_pos_found > 0:
    print(f"Check:    Error detected at position {error_pos_found}")
    print(f"Corrected:{''.join(map(str, corrected))}")
    if error_pos_found == error_pos_actual and corrected == encoded:
        print("          (Correction Successful)")
    else:
        print("          (Correction FAILED!)")
elif error_pos_found == 0:
     print("Check:    No error detected.")
elif error_pos_found == -1:
     print("Check:    Double error detected (uncorrectable).")

Data:     11001100100
Encoded:  0010100111001000
Corrupted:0110100111001000 (err @2)
Check:    Error detected at position 2
Corrected:0010100111001000
          (Correction Successful)


In [8]:
from functools import reduce
import random
# Random bit string
X = [random.randint(0,1) for i in range(16)]

# Set 0 and poewrs of 2 to 0
X[0] = 0
for i in range (4):
    X[2**i] = 0

# compute blockwsise paraties with xor func
paraties = bin(reduce(lambda x,y: x^y, [i for i,bit in enumerate(X) if bit]))[2:]
paraties = [int(j) for j in paraties][::-1]

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2583917726.py, line 1)